## HYBRID SEARCH

In [9]:
!pip uninstall pinecone-client

In [1]:
!pip install --upgrade --quiet pinecone pinecone-text pinecone-notebooks


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
index_name = "hybrid-search-langchain-pinecone"

## Initialize the pinecone client 
pc = Pinecone(api_key=PINECONE_API_KEY)

## Create the index if it doesn't exist
if not pc.has_index(name=index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [6]:
index = pc.Index(index_name)
index

In [7]:
## Vector embedding and sparse matrix 
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from pinecone_text.sparse import BM25Encoder 

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [10]:
sentences = [
    "In 2023 I Visited Paris ",
    "In 2022 I visited New York",
    "In 2021, I visited New Orleans"
]

## tf-idf values on these sentences 
bm25_encoder.fit(sentences)

## store the values to a json file 
bm25_encoder.dump("bm25_values.json")

## load your BM25Encoder Object (load is an instance method, not a class method)
bm25_encoder = BM25Encoder()
bm25_encoder.load("bm25_values.json")





100%|██████████| 3/3 [00:00<00:00, 6998.28it/s]


In [18]:
retriever =  PineconeHybridSearchRetriever(
    index = index,
    sparse_encoder = bm25_encoder,
    embeddings=embeddings
    
)

In [19]:
retriever


PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000024627AC7310>, index=<pinecone.db_data.index.Index object at 0x00000245B71A4B80>)

In [21]:
retriever.add_texts(
    [
        "In 2023 I Visited Paris ",
        "In 2022 I visited New York",
        "In 2021, I visited New Orleans"
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


In [22]:
retriever.invoke("what city did i visit last?")

[Document(metadata={'score': 0.287524343}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.262057424}, page_content='In 2022 I visited New York'),
 Document(metadata={'score': 0.232148856}, page_content='In 2023 I Visited Paris ')]